<a href="https://colab.research.google.com/github/Gonzalosd0/AI/blob/main/Modelo_CVaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# -------------------------------
# 🔷 MÓDULO INSTITUCIONAL CVaR COMPLETO
# -------------------------------

def simulate_returns_bootstrap(historical_returns, sims=10000, days=1, seed=42):
    """Bootstrap de retornos históricos."""
    np.random.seed(seed)
    sampled = np.random.choice(historical_returns.shape[0], size=(sims, days), replace=True)
    return historical_returns[sampled].sum(axis=1)

def calculate_cvar(returns, alpha=0.95):
    """Calcula el CVaR empírico."""
    sorted_returns = np.sort(returns)
    var_index = int((1 - alpha) * len(sorted_returns))
    return -np.mean(sorted_returns[:var_index])

def cvar_objective(weights, simulated_returns, alpha=0.95):
    """Función objetivo que minimiza el CVaR para una cartera dada."""
    portfolio_returns = simulated_returns @ weights
    return calculate_cvar(portfolio_returns, alpha)

def max_drawdown(returns):
    """Cálculo de Maximum Drawdown de una serie de retornos acumulados."""
    cumulative = np.cumprod(1 + returns)
    peak = np.maximum.accumulate(cumulative)
    drawdown = (cumulative - peak) / peak
    return drawdown.min()

def cvar_optimizer(historical_returns, alpha=0.95, days=1, sims=10000,
                   max_vol=None, max_dd=None, bounds=None, verbose=False):
    """
    Optimización de CVaR con restricciones adicionales:
    - Volatilidad máxima anualizada
    - Drawdown máximo
    """
    n_assets = historical_returns.shape[1]
    asset_names = historical_returns.columns.tolist()
    returns_matrix = historical_returns.to_numpy()

    # Simulación bootstrap
    sim_returns = np.zeros((sims, n_assets))
    for i in range(n_assets):
        sim_returns[:, i] = simulate_returns_bootstrap(returns_matrix[:, i], sims, days)

    cov_matrix = historical_returns.cov().values
    if bounds is None:
        bounds = [(0.0, 1.0) for _ in range(n_assets)]
    initial_guess = np.ones(n_assets) / n_assets

    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    if max_vol:
        daily_vol = max_vol / np.sqrt(252)
        constraints.append({
            'type': 'ineq',
            'fun': lambda w: daily_vol - np.sqrt(w.T @ cov_matrix @ w)
        })

    if max_dd:
        def dd_constraint(w):
            simulated = sim_returns @ w
            return -max_drawdown(simulated) - max_dd  # debe ser ≥ 0
        constraints.append({'type': 'ineq', 'fun': dd_constraint})

    result = minimize(cvar_objective,
                      initial_guess,
                      args=(sim_returns, alpha),
                      method='SLSQP',
                      bounds=bounds,
                      constraints=constraints)

    weights = result.x
    final_cvar = calculate_cvar(sim_returns @ weights, alpha)
    final_vol = np.sqrt(weights.T @ cov_matrix @ weights) * np.sqrt(252)
    final_dd = max_drawdown(sim_returns @ weights)

    if verbose:
        print(f"CVaR {int(alpha*100)}%: {-final_cvar:.4%}")
        print(f"Vol anualizada: {final_vol:.2%}")
        print(f"Máx. Drawdown: {final_dd:.2%}")

    results_df = pd.DataFrame({
        'Asset': asset_names,
        'Weight (%)': (weights * 100).round(2)
    })
    results_df.loc['Total'] = ['Total', results_df['Weight (%)'].sum()]

    return results_df, -final_cvar, final_vol, final_dd

# -------------------------------
# EJEMPLO DE USO CON DATOS SIMULADOS
# -------------------------------

np.random.seed(42)
dates = pd.date_range(start="2023-01-01", periods=500, freq="B")
assets = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]
returns = pd.DataFrame(index=dates)
for asset in assets:
    returns[asset] = np.random.normal(0.0005, 0.02, len(dates))

# Ejecutar optimización institucional con CVaR 95%, Vol máx 18%, DD máx 20%
cvar_df, cvar_val, vol_val, dd_val = cvar_optimizer(
    historical_returns=returns,
    alpha=0.95,
    days=1,
    sims=10000,
    max_vol=0.18,
    max_dd=0.20,
    verbose=True
)

cvar_df, round(cvar_val, 4), round(vol_val, 4), round(dd_val, 4)



CVaR 95%: -1.7664%
Vol anualizada: 13.90%
Máx. Drawdown: -20.00%


(       Asset  Weight (%)
 0       AAPL       20.18
 1       MSFT        9.46
 2       GOOG       21.25
 3       AMZN       24.06
 4       META       25.06
 Total  Total      100.01,
 np.float64(-0.0177),
 np.float64(0.139),
 np.float64(-0.2))

In [ ]:
🔧 ¿Qué puedes modificar y cómo afecta?
1. Horizonte temporal (days)
days = 1        # diario
days = 21       # mensual
days = 63       # trimestral

2. Nivel de confianza (alpha)
python
Copiar
Editar
alpha = 0.95    # estándar
alpha = 0.99    # para instituciones conservadoras
📉 Aumentar alpha captura riesgos más extremos.

Muy útil en cumplimiento de riesgo extremo (banca privada, ALM, seguros).

3. Método de simulación
Actualmente: bootstrap sobre retornos históricos:

python
Copiar
Editar
simulate_returns_bootstrap(...)
Puedes sustituir por:
GARCH: con librería arch, para modelar volatilidad dinámica.

CóPULAS: para simular dependencias entre activos en crisis.

Monte Carlo paramétrico: con media, covarianza y ruido controlado.

👍 Esto te da control sobre colas, correlaciones y shocks sistémicos.


4. Restricción de volatilidad (max_vol)
python
Copiar
Editar
max_vol = 0.18  # anualizada
🎯 Útil para construir carteras ajustadas a perfil de cliente MIFID.

Se convierte internamente a diaria y limita el riesgo global.

Puedes quitarla con max_vol=None.

5. Restricción de drawdown (max_dd)
python
Copiar
Editar
max_dd = 0.20   # tolerancia máxima a caídas extremas
🚨 Protege contra pérdida máxima en simulación.

Especialmente útil en estrategias de preservación de capital.

Puedes desactivarla con max_dd=None.

6. Restricciones de peso (bounds)
python
Copiar
Editar
bounds = [(0.05, 0.25) for _ in range(n_assets)]
Controla diversificación: evita activos con peso nulo o excesivo.

Requiere si estás sujeto a normas UCITS, ESG, sectoriales o límites internos.

7. Número de simulaciones (sims)
python
Copiar
Editar
sims = 10000  # recomendable mínimo 5,000
A mayor número, mejor estimación de CVaR, pero más lento.

Puedes hacerlo dependiente del horizonte o tamaño de cartera.

8. Reemplazar función objetivo
Puedes mezclar CVaR con otras métricas:

python
Copiar
Editar
def objective(w):
    cvar = ...
    tracking_error = ...
    return 0.9 * cvar + 0.1 * tracking_error



🧠 ¿Qué estrategia seguirías?
Te recomiendo crear versiones preconfiguradas por perfil de riesgo:

Perfil	α	Vol máx	DD máx	Bounds
Conservador	99%	10%	15%	(0.05, 0.25)
Moderado	95%	15%	20%	(0.05, 0.35)
Agresivo	90%	25%	30%	(0.0, 0.50)